# Backtesting fee analysis

This is an example notebook how different fee models
can be implemented in backtesting.

Some highlights of this notebook:

- Uses synthetic data with a fixed asset price
    - Makes it easier to manual confirm correct price calculations
- Show how to manually set up a fee model for a trading pair
- Trade 1h cycles, do one trade at every midnight, run for 2 months to generate a visualisation
- Shows a fee calculation based on Uniswap v2 LP fee

*Note that if you are running in this notebook [in PyCharm you may encounter "IOPub data rate exceeded" error that needs a workaround](https://stackoverflow.com/a/75184953/315168).*

## Strategy logic and trade decisions

Because we are interested in fees only,
we do random sized buy and sell every second hour.

In [10]:
import datetime
import logging
from typing import List, Dict

import pandas as pd

from tradingstrategy.chain import ChainId
from tradingstrategy.timebucket import TimeBucket
from tradingstrategy.universe import Universe

from tradeexecutor.strategy.cycle import CycleDuration
from tradeexecutor.strategy.strategy_module import TradeRouting, ReserveCurrency
from tradeexecutor.state.trade import TradeExecution
from tradeexecutor.strategy.pricing_model import PricingModel
from tradeexecutor.strategy.pandas_trader.position_manager import PositionManager
from tradeexecutor.state.state import State



def decide_trades(
        timestamp: pd.Timestamp,
        universe: Universe,
        state: State,
        pricing_model: PricingModel,
        cycle_debug_data: Dict) -> List[TradeExecution]:

    # The pair we are trading
    pair = universe.pairs.get_single()

    # Create a position manager helper class that allows us easily to create
    # opening/closing trades for different positions
    position_manager = PositionManager(timestamp, universe, state, pricing_model)

    amount = random.choice([250, 500, 750, 1000])

    # Trade every midnight
    trades = []
    if timestamp.hour == 0:
        if not position_manager.is_any_open():
            # Buy
            trades += position_manager.open_1x_long(pair, amount)
        else:
            # Sell
            trades += position_manager.close_all()

    return trades

# Defining trading universe

We create a trading universe that has ETH/USD asset with a fixed $1000 price.

The pair has fixed 0.3% fee tier. We generate data for 8 weeks.

In [11]:

import random
from tradeexecutor.state.identifier import AssetIdentifier, TradingPairIdentifier
from tradingstrategy.candle import GroupedCandleUniverse
from tradeexecutor.testing.synthetic_ethereum_data import generate_random_ethereum_address
from tradeexecutor.testing.synthetic_exchange_data import generate_exchange
from tradeexecutor.testing.synthetic_price_data import generate_ohlcv_candles
from tradeexecutor.strategy.trading_strategy_universe import TradingStrategyUniverse, \
    create_pair_universe_from_code

def create_trading_universe() -> TradingStrategyUniverse:

    # Set up fake assets
    mock_chain_id = ChainId.ethereum
    mock_exchange = generate_exchange(
        exchange_id=random.randint(1, 1000),
        chain_id=mock_chain_id,
        address=generate_random_ethereum_address())
    usdc = AssetIdentifier(ChainId.ethereum.value, generate_random_ethereum_address(), "USDC", 6, 1)
    weth = AssetIdentifier(ChainId.ethereum.value, generate_random_ethereum_address(), "WETH", 18, 2)
    weth_usdc = TradingPairIdentifier(
        weth,
        usdc,
        generate_random_ethereum_address(),
        mock_exchange.address,
        internal_id=random.randint(1, 1000),
        internal_exchange_id=mock_exchange.exchange_id,
        fee=0.003,
    )

    pair_universe = create_pair_universe_from_code(mock_chain_id, [weth_usdc])

    candles = generate_ohlcv_candles(
        TimeBucket.h1,
        start=datetime.datetime(2021, 6, 1),
        end=datetime.datetime(2021, 8, 1),
        pair_id=weth_usdc.internal_id,
        start_price=1000,
        daily_drift=(1, 1),
        high_drift=1,
        low_drift=1,
    )
    candle_universe = GroupedCandleUniverse.create_from_single_pair_dataframe(candles)

    universe = Universe(
        time_bucket=TimeBucket.h1,
        chains={mock_chain_id},
        exchanges={mock_exchange},
        pairs=pair_universe,
        candles=candle_universe,
    )

    return TradingStrategyUniverse(universe=universe, reserve_assets=[usdc])


## Examining the generated data

Before starting the backtest, do a smoke check that our trading universe looks correct.

In [12]:
universe = create_trading_universe()

start_at, end_at = universe.universe.candles.get_timestamp_range()
print(f"Our universe has synthetic data for the period {start_at} - {end_at}")
pair = universe.get_single_pair()
candles = universe.universe.candles.get_samples_by_pair(pair.internal_id)
min_price = candles["close"].min()
max_price = candles["close"].max()
print(f"We trade {pair}")
print(f"Price range is {min_price} - {max_price}")


Our universe has synthetic data for the period 2021-06-01 00:00:00 - 2021-07-31 23:00:00
We trade <Pair WETH-USDC at 0x0D4Da698e10aCE06CEFcEa4b50B2a53986fEA7fD (0.3000% fee) on exchange 0x2813E651a102166b5beBBBF91dAFc64099C0645d>
Price range is 1000.0 - 1000.0


## Running the backtest

In [13]:
from tradeexecutor.testing.synthetic_exchange_data import generate_simple_routing_model
from tradeexecutor.backtest.backtest_runner import run_backtest_inline

routing_model = generate_simple_routing_model(universe)

state, universe,    debug_dump = run_backtest_inline(
    name="Synthetic random data backtest",
    start_at=start_at.to_pydatetime(),
    end_at=end_at.to_pydatetime(),
    client=None,
    cycle_duration=CycleDuration.cycle_1h,
    decide_trades=decide_trades,
    universe=universe,
    initial_deposit=10_000,
    reserve_currency=ReserveCurrency.usdc,
    trade_routing=TradeRouting.user_supplied_routing_model,
    routing_model=routing_model,
    log_level=logging.WARNING,
)


  0%|          | 0/5266800 [00:00<?, ?it/s]

## Examine backtest results

Examine `state` that contains all actions the trade executor took.

We plot out a chart that shows
- When the strategy made buys or sells
- Mouse hovers show the fees paid

In [14]:
from tradeexecutor.visual.single_pair import visualise_single_pair

figure = visualise_single_pair(state, universe.universe.candles)

figure.show()

In [15]:
from tradeexecutor.analysis.trade_analyser import build_trade_analysis

analysis = build_trade_analysis(state.portfolio)

### Strategy summary

Overview of strategy performance.

We manually check that fees were correctly calculated.



In [16]:
from IPython.core.display_functions import display

summary = analysis.calculate_summary_statistics()

with pd.option_context("display.max_row", None):
    display(summary.to_dataframe())

,0
Trading period length,60 days
Return %,-1.17%
Annualised return %,-7.11%
Cash at start,"$10,000.00"
Value at end,"$9,883.18"
Trade win percent,0.00%
Total trades done,30
Won trades,0
Lost trades,30
Stop losses triggered,0


### Trade timeline

Display all positions and how much profit they made.
Manually check the total swap fees column that it looks correct.


In [17]:
from tradeexecutor.analysis.trade_analyser import expand_timeline

timeline = analysis.create_timeline()

expanded_timeline, apply_styles = expand_timeline(
        universe.universe.exchanges,
        universe.universe.pairs,
        timeline)

# Do not truncate the row output
with pd.option_context("display.max_row", None):
    display(apply_styles(expanded_timeline))


Remarks,Opened at,Duration,Exchange,Base asset,Quote asset,Position max size,PnL USD,PnL %,Open price USD,Close price USD,Trade count,Total swap fees
,2021-06-01,1 days,,WETH,USDC,$750.00,$-4.49,-0.60%,"$1,006.018054",$999.991000,2,$4.49
,2021-06-03,1 days,,WETH,USDC,"$1,000.00",$-5.99,-0.60%,"$1,006.018054",$999.991000,2,$5.98
,2021-06-05,1 days,,WETH,USDC,$750.00,$-4.49,-0.60%,"$1,006.018054",$999.991000,2,$4.49
,2021-06-07,1 days,,WETH,USDC,$250.00,$-1.50,-0.60%,"$1,006.018054",$999.991000,2,$1.50
,2021-06-09,1 days,,WETH,USDC,$750.00,$-4.49,-0.60%,"$1,006.018054",$999.991000,2,$4.49
,2021-06-11,1 days,,WETH,USDC,$250.00,$-1.50,-0.60%,"$1,006.018054",$999.991000,2,$1.50
,2021-06-13,1 days,,WETH,USDC,$500.00,$-3.00,-0.60%,"$1,006.018054",$999.991000,2,$2.99
,2021-06-15,1 days,,WETH,USDC,$750.00,$-4.49,-0.60%,"$1,006.018054",$999.991000,2,$4.49
,2021-06-17,1 days,,WETH,USDC,$750.00,$-4.49,-0.60%,"$1,006.018054",$999.991000,2,$4.49
,2021-06-19,1 days,,WETH,USDC,$250.00,$-1.50,-0.60%,"$1,006.018054",$999.991000,2,$1.50


## Finishing notes

Print out a line to signal the notebook finished the execution successfully.

In [18]:
print("All ok")

All ok
